In [1]:
import numpy as np
from numpy import loadtxt
import fitsio
import healpy as hp
import healsparse as hsp
import skyproj
import matplotlib.pyplot as plt
import Config
from os import listdir
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
Class_Cutoff = 2.5

In [3]:
def findMatches(angleCutoff, RASource, DECSource, RAMatchCatalog, DECMatchCatalog, nthneighbor=1):
    c = SkyCoord(ra=RASource*u.degree, dec=DECSource*u.degree)
    catalog = SkyCoord(ra=RAMatchCatalog*u.degree, dec=DECMatchCatalog*u.degree)
    idx, d2d, d3d = c.match_to_catalog_sky(catalog, nthneighbor=nthneighbor)
    matches = d2d < angleCutoff
    return matches, d2d

In [4]:
res = 512

likeDir = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/MaximumLikelihood/'

minGR = -0.3
maxGR = 1

magBins = [20, 22, 23, 24, 25]

pixFile = likeDir + 'PixAndConds/Valid_'+str(res)+'_Pixels.fits'
validPix = fitsio.read(pixFile)['PIXEL']
pixCheck = np.full(12*(res**2), False, dtype = bool)
pixCheck[validPix] = True

caliDir = likeDir + 'Calibration/'
wideStarPos = [caliDir + 'Class_'+str(Class_Cutoff)+'_Star_Pos_<20.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Star_Pos_20-22.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Star_Pos_22-23.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Star_Pos_23-24.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Star_Pos_24-25.fits']
wideGalaPos = [caliDir + 'Class_'+str(Class_Cutoff)+'_Gala_Pos_<20.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Gala_Pos_20-22.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Gala_Pos_22-23.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Gala_Pos_23-24.fits', caliDir + 'Class_'+str(Class_Cutoff)+'_Gala_Pos_24-25.fits']

starDir = likeDir + 'Stars/'
starProbFiles = [starDir + 'Class_'+str(Class_Cutoff)+'_Star_Prob_<20.fits', starDir + 'Class_'+str(Class_Cutoff)+'_Star_Prob_20-22.fits', starDir + 'Class_'+str(Class_Cutoff)+'_Star_Prob_22-23.fits', starDir + 'Class_'+str(Class_Cutoff)+'_Star_Prob_23-24.fits', starDir + 'Class_'+str(Class_Cutoff)+'_Star_Prob_24-25.fits']

galaDir = likeDir + 'Galaxies/'
galaProbFiles = [galaDir + 'Class_'+str(Class_Cutoff)+'_Gala_Prob_<20.fits', galaDir + 'Class_'+str(Class_Cutoff)+'_Gala_Prob_20-22.fits', galaDir + 'Class_'+str(Class_Cutoff)+'_Gala_Prob_22-23.fits', galaDir + 'Class_'+str(Class_Cutoff)+'_Gala_Prob_23-24.fits', galaDir + 'Class_'+str(Class_Cutoff)+'_Gala_Prob_24-25.fits']

In [5]:
starAdjustments = []
galaAdjustments = []

for i in np.arange(len(wideStarPos)):
    deepCols  = ['KNN_CLASS', 'RA', 'DEC', 'BDF_MAG_DERED_G', 'BDF_MAG_DERED_R', 'MASK_FLAGS', 'MASK_FLAGS_NIR']
    deepRA = np.array([])
    deepDEC = np.array([])
    deepG = np.array([])
    deepGR = np.array([])
    deepMasks = np.array([])
    deepClass = np.array([])
    deepFlag = np.array([])
    deepFlagNir = np.array([])

    for j in (np.arange(5) + 1):
        deepFile = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_00000' + str(j) + '.fits'

        deepData = fitsio.read(deepFile, columns = deepCols)

        deepRA = np.append(deepRA, deepData['RA'])
        deepDEC = np.append(deepDEC, deepData['DEC'])
        deepG = np.append(deepG, deepData['BDF_MAG_DERED_G'])
        deepGR = np.append(deepGR, deepData['BDF_MAG_DERED_G'] - deepData['BDF_MAG_DERED_R'])
        deepClass = np.append(deepClass, deepData['KNN_CLASS'])
        deepFlag = np.append(deepFlag, deepData['MASK_FLAGS'])
        deepFlagNir = np.append(deepFlagNir, deepData['MASK_FLAGS_NIR'])

    if i == 0:
        cutIndices = np.where((deepFlag == 0) &
                              (deepFlagNir == 0) &
                              # (deepGR <= maxGR) &
                              # (deepGR >= minGR) &
                              # (deepG <= magBins[i]) &
                              (deepRA < 120) &
                              (deepClass > 0) &
                              (deepClass <= 3))[0]
    else:
        cutIndices = np.where((deepFlag == 0) &
                              (deepFlagNir == 0) &
                              # (deepGR <= maxGR) &
                              # (deepGR >= minGR) &
                              # (deepG <= magBins[i]) &
                              # (deepG > magBins[i - 1]) &
                              (deepRA < 120) &
                              (deepClass > 0) &
                              (deepClass <= 3))[0]

    deepG = deepG[cutIndices]
    deepClass = deepClass[cutIndices]
    deepRA = deepRA[cutIndices]
    deepDEC = deepDEC[cutIndices]

    deepPix = np.unique(hp.ang2pix(res, deepRA, deepDEC, lonlat = True, nest = True))
    print(len(np.where(deepClass == 3)[0]))
    print(len(deepClass))

    deepPixCheck = np.full(12*(res**2), False, dtype = bool)
    deepPixCheck[deepPix] = True
    
    allStarData = fitsio.read(wideStarPos[i])
    allStarRA = allStarData['RA']
    allStarDEC = allStarData['DEC']

    allGalaData = fitsio.read(wideGalaPos[i])
    allGalaRA = allGalaData['RA']
    allGalaDEC = allGalaData['DEC']
    
    deepStarMatches, _ = findMatches(0.5*u.arcsec, deepRA, deepDEC, allStarRA, allStarDEC)
    deepGalaMatches, _ = findMatches(0.5*u.arcsec, deepRA, deepDEC, allGalaRA, allGalaDEC)

    matchedDeepStarRA = deepRA[deepStarMatches]
    matchedDeepStarDEC = deepDEC[deepStarMatches]
    matchedDeepStarClass = deepClass[deepStarMatches]

    matchedDeepGalaRA = deepRA[deepGalaMatches]
    matchedDeepGalaDEC = deepDEC[deepGalaMatches]
    matchedDeepGalaClass = deepClass[deepGalaMatches]
    
    TSPIX = hp.ang2pix(res, matchedDeepStarRA[np.where(matchedDeepStarClass == 2)[0]], matchedDeepStarDEC[np.where(matchedDeepStarClass == 2)[0]], lonlat = True, nest = True)
    FSPIX = hp.ang2pix(res, matchedDeepStarRA[np.where(matchedDeepStarClass == 1)[0]], matchedDeepStarDEC[np.where(matchedDeepStarClass == 1)[0]], lonlat = True, nest = True)

    TGPIX = hp.ang2pix(res, matchedDeepGalaRA[np.where(matchedDeepGalaClass == 1)[0]], matchedDeepGalaDEC[np.where(matchedDeepGalaClass == 1)[0]], lonlat = True, nest = True)
    FGPIX = hp.ang2pix(res, matchedDeepGalaRA[np.where(matchedDeepGalaClass == 2)[0]], matchedDeepGalaDEC[np.where(matchedDeepGalaClass == 2)[0]], lonlat = True, nest = True)
    
    starCorrProb = np.clip(fitsio.read(starProbFiles[i])['SIGNAL'], 0, 1)
    fullStarProb = np.full(12*(res**2), hp.UNSEEN)
    fullStarProb[validPix] = starCorrProb

    galaCorrProb = np.clip(fitsio.read(galaProbFiles[i])['SIGNAL'], 0, 1)
    fullGalaProb = np.full(12*(res**2), hp.UNSEEN)
    fullGalaProb[validPix] = galaCorrProb
    
    print(len(TSPIX))
    print(len(FGPIX))
    print(len(TSPIX) / (np.sum(fullStarProb[TSPIX[np.where(pixCheck[TSPIX])[0]]]) + np.sum(fullStarProb[FGPIX[np.where(pixCheck[FGPIX])[0]]])))
    starAdjustments.append(len(TSPIX) / (np.sum(fullStarProb[TSPIX[np.where(pixCheck[TSPIX])[0]]]) + np.sum(fullStarProb[FGPIX[np.where(pixCheck[FGPIX])[0]]])))
    
    print(len(TGPIX))
    print(len(FSPIX))
    print(len(TGPIX) / (np.sum(fullGalaProb[TGPIX[np.where(pixCheck[TGPIX])[0]]]) + np.sum(fullGalaProb[FSPIX[np.where(pixCheck[FSPIX])[0]]])))
    galaAdjustments.append(len(TGPIX) / (np.sum(fullGalaProb[TGPIX[np.where(pixCheck[TGPIX])[0]]]) + np.sum(fullGalaProb[FSPIX[np.where(pixCheck[FSPIX])[0]]])))
    
caliTable = Table()
caliTable['STAR'] = starAdjustments
caliTable['GALA'] = galaAdjustments
caliTable.write(caliDir + str(Class_Cutoff) + '_Class_Calibrations.fits', overwrite = True)

1
1130504
762
11
0.9889323760503823
1082
87
0.9878486024677244
1
1130504
1928
46
0.993348122668869
8232
429
0.9942080436668008


KeyboardInterrupt: 